In [1]:
import torch
import json
import re

from datasets import load_dataset
from tqdm import tqdm
from data.discovery_con import LABELS

from transformers import AutoModelForCausalLM, AutoTokenizer

In [2]:
# dataset = load_dataset('discofuse', 'discofuse-wikipedia', split='train[:10%]')

In [3]:
# dataset["train"][0]

In [4]:
# def filtering(example):
#     if "connective_string" in example: return True
#     else: return False

In [5]:
# dataset["train"].filter(filtering)

In [2]:
dataset = load_dataset('data/discovery.py', 'discovery')

Reusing dataset discovery (/home/nlp/.cache/huggingface/datasets/discovery/discovery/1.0.0/f08ced5950fb93854c70d20fc70d1583766d7219cda67e65d197dfe9ec3775ca)


In [48]:
discovery_train_ds = dataset["train"]
discovery_valid_ds = dataset["validation"]
discovery_test_ds = dataset["test"]

In [49]:
tokenizer.add_special_tokens({'pad_token': '[PAD]'})

0

In [58]:
tokenizer.decode(tokenizer(discovery_train_ds[0]['sentence1']).input_ids)

'He helped to found the Mexican American Legal Defense and Educational Fund (MALDEF) and Texas Rural Legal Aid during the 1940s.'

In [40]:
model = AutoModelForCausalLM.from_pretrained('/home/nlp/apex/experiment/ctrl/checkpoint-5000')

In [55]:
tokenizer = AutoTokenizer.from_pretrained('/home/nlp/apex/experiment/ctrl/checkpoint-5000', padding_side='right',
                                         max_length=64)

In [42]:
# training gpt2 on discofuse and discovery doesn't help
# training gpt2 on seq class didn't help
# Control codes; baby

In [43]:
class DiscoveryDatasetAF():
    
    def __init__(self, dataset, labels, tokenizer, model):
        self.dataset = dataset
        self.labels = labels
        self.tokenizer = tokenizer
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(self.device)
        
    def get_sentence_as_context(self, idx, sentence_order):
        context = self.dataset[idx]
        tokenized_context = self.tokenizer(self.labels[context['label']] + ' ' + context[sentence_order], return_tensors="pt")
        return tokenized_context, self.labels[context['label']]
    
    def generate_synthetic_options(self, idx, context='sentence1'):
        tokenized_context, marker = self.get_sentence_as_context(idx, context)
        tokenized_context = tokenized_context['input_ids'].to(self.device)
        input_ids, len_context = tokenized_context, len(self.dataset[idx][context])
        outputs = model.generate(input_ids,
                                do_sample = True, 
                                max_length = 50, 
                                top_k = 50, 
                                top_p = 0.95, 
                                num_return_sequences = 3
                                )
        example = {}
        example['ground_truth'] = self.dataset[idx]['sentence2']
        for i, sample_output in enumerate(outputs):
            uncleaned_text = tokenizer.decode(sample_output, skip_special_tokens=True)[len_context:]
            cleaned_text = uncleaned_text.replace('\n', '').replace('\xa0', '').replace('\\', '').replace(marker, '').strip()
            example['option_' + str(i)] = cleaned_text
        return example

In [44]:
discovery_ds = DiscoveryDatasetAF(discovery_train_ds, LABELS, tokenizer, model)

In [45]:
synthetic_dataset = []

for i in tqdm(range(len(discovery_train_ds))):
    example = {}
    values = discovery_train_ds[i]
    example['context'] = values['sentence1']
    example['marker'] = LABELS[values['label']]
    generated_options = discovery_ds.generate_synthetic_options(i)
    example.update(generated_options)
    synthetic_dataset.append(example)

  0%|          | 84/1566000 [01:57<607:37:18,  1.40s/it]


KeyboardInterrupt: 

In [47]:
synthetic_dataset[1]

{'context': '"Then click the``Paper Clip""button in the Note Editor, and select the image file from your hard drive in the file dialog to insert the image in to the note."',
 'marker': 'alternately',
 'ground_truth': 'You can use any handheld device that runs the iOS or Android operating systems to take a picture and insert the picture immediately.',
 'option_0': 'o the note." Note: if you find that the Notebook is not accepting a file, remove',
 'option_1': 'o the note." or gp images may be provided in a variety of other format than',
 'option_2': 'o the note." you may select a high quality image from your computer in the image preview'}

In [83]:
with open('data/synthetic_discovery_100.json', 'w') as fout:
    json.dump(synthetic_dataset , fout)

In [8]:
discovery_train_ds.column_names[0]

'idx'

In [ ]:
verb
coreference

In [56]:
from transformers import GPT2Config, GPT2Tokenizer, GPT2ForSequenceClassification

In [58]:
config = GPT2Config.from_pretrained('microsoft/DialoGPT-large', num_labels = len(LABELS))
tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialoGPT-large')
model = GPT2ForSequenceClassification.from_pretrained('microsoft/DialoGPT-large', config=config)

Some weights of the model checkpoint at microsoft/DialoGPT-large were not used when initializing GPT2ForSequenceClassification: ['lm_head.weight']
- This IS expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing GPT2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at microsoft/DialoGPT-large and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [66]:
label = torch.tensor(example['label'])

In [54]:
inputs = tokenizer(example['sentence1'], return_tensors="pt")

In [69]:
outputs = model(**inputs, labels=label)

In [73]:
torch.argmax(outputs[1], axis=1)

tensor([82])